# Парсинг pdf-документов (выделение текста)

In [1]:
import io
import re
import os, wget
import requests
import glob
from tqdm.notebook import tqdm

from tika import parser

UNIVS = ['АлтГУ', 'АлтГТУ', 'ВолГУ', 'ИМСИТ']
CURRPATH = os.getcwd()
DATAPATH = '\\'.join(CURRPATH.split('\\')[:-1]) + '\\new-data'
FILESPATH = DATAPATH + '\\new-files'

try:
    os.makedirs(FILESPATH)
except:
    pass

In [2]:
regex_general = [
    r'Форма обучения',
    r'Объем дисциплины',
    r'Форма промежуточной аттестации',
    r'Контрольная работа',
    r'Зачет\W', r'Экзамен\W', r'Очная', r'Заочная',
    r'стр[.] ?\d\d{0,2}-?\d{0,3}',
    r'cтр[.] ?\d\d{0,2}-?\d{0,3}', # eng c
    r'Тема \d\d?[.]?', r'Модуль \d\d?[.]?', r'Модуль [IVXCDL]*', r'\d[.]',
    r'\s\d\d?',
    r'Семестр', r'Раздел \w{1,3}\W', 
    r'https?://.*?[.]com/?',
    r'https?://.*?[.]ru/?',
    r'https?://.*?[.]org/?',
    r'www.*?[.]com/?',
    r'www.*?[.]ru/?',
    r'www.*?[.]org/?',
    r'\[.*?\]',
    r'\(\d{1,3}\)',
    r'\(\d{1,3}, \d{1,3}\)',
    r'\s\-', r'\-\s',
    r'[.,:;\"\'«»\(\)*!?\/\\–—№]',
]

regex_imsit = [    
    r'.*Цель *?и *?задачи',
    r'Краткая *?характеристика',
    r'Содержание *?дисциплины', 
    r'учебной *?дисциплины',
    r'Место *?дисциплины *?в *?структуре ОПОП.*\(основные блоки, темы\)',
    r'Знания, *?умения *?и *?навыки',
    r'Формы *?проведения.*',
    r'Компетенции',
    r'Задачи *?дисциплины',
    r'изучения *?дисциплины',
    
]

regex_volgu = [    
    r'.*4. Цель и задачи изучения дисциплины',
    r'Цель *?изучения *?дисциплины',
    r'Задачи дисциплины',
    r'Место дисциплины в структуре ОПОП.*\d. Содержание дисциплины',
    r'Выпускник *?должен *?обладать *?следующими *?общепрофессиональными *?компетенциями',
    r'Выпускник *?должен *?обладать *?следующими *?универсальными *?компетенциями',
    r'Процесс  изучения  дисциплины  направлен.*ФГОС ВО',
    r'Требования к результатам освоения дисциплины',
    r'Студент должен',
]

regex_altgtu = [
    r'Разработал:.*',
    r'.*В результате освоения дисциплины обучающийся должен обладать следующими компетенциями:',
    r'Контрольный опрос и тест по модулю',
    r'Синтаксис текста',
]

regex_altgu = [
    r'.*Цель изучения дисциплины',
    r'Форма промежуточной аттестации.*',
    r'Место дисциплины в учебном плане .*?  Формируемые',
    r'Используемые информационные, инструментальные и программные средства.*',
    r'Виды учебной работы.*',
    r'Целями изучения дисциплины являются',
]

REGEXES = {
    'ИМСИТ' : regex_imsit,
    'ВолГУ' : regex_volgu,
    'АлтГТУ' : regex_altgtu,
    'АлтГУ' : regex_altgu,
}

In [3]:
def parse(file):
    raw = parser.from_file(file)
    text = raw['content'].encode("ansi", "ignore").decode('ansi')
    return text.replace('ё', 'е').replace('-\n\n', '').replace('-\t', ' ').replace('\n', ' ').replace('\t', ' ')


def brush(text, regex):
    for reg in regex:
        text = re.sub(reg, ' ', text, flags=re.IGNORECASE)
    for reg in regex_general:
        text = re.sub(reg, ' ', text, flags=re.IGNORECASE)
    return text


def clean_text(text, regex):
    text = brush(text, regex)
    text = re.sub(r'[().,:;]', '', text)
    text = re.sub(r'[ОПКИДУ]{2,3}[- ]?\d[.]?\d?', '', text)
    text = re.sub(r'\ {2,}', ' ', text).lstrip().rstrip()
    return text


def save(file, text, split_by='\\'):
    splitted = file.split(split_by)
    splitted[-1] = splitted[-1].replace('.pdf', '.txt')
    new_path = FILESPATH + '\\' + '\\'.join(splitted[-3:-1])
    
    try:
        os.makedirs(new_path)
    except:
        pass
    
    new_path += '\\' + splitted[-1]
    
    file_ = open(new_path, "w+")
    file_.write(text)
    file_.close()

In [4]:
for U in tqdm(UNIVS):
    regex = REGEXES[U]
    for file in glob.glob(DATAPATH + '\{}\*\*.pdf'.format(U)):
        try:
            text = parse(file)
        except Exception as e:
            print('parsing error for {}:'.format('\\'.join(file.split('\\')[-3:])), e)
            print()
            continue
        try:
            text = clean_text(text, regex)
        except Exception as e:
            print('cleaning error for {}:'.format('\\'.join(file.split('\\')[-3:])), e)
            print()
            continue
        try:
            save(file, text)
        except Exception as e:
            print('saving error for {}:'.format('\\'.join(file.split('\\')[-3:])), e)
            print()
            continue

  0%|          | 0/4 [00:00<?, ?it/s]